In [1]:
import aocd
puzzle = aocd.get_puzzle(year=2024, day=9)

puzzle.examples[0].input_data

'2333133121414131402'

In [2]:
import numpy as np
import itertools as it

In [3]:
def parse_input(input_data):
    sizes = np.array(list(map(int, input_data)))
    full_width = sizes.sum()
    memory = np.full(full_width, -1)

    is_file = True
    file_id = 0
    position = 0

    files, gaps = {}, {}
    for size in sizes:
        if is_file:
            memory[position : position + size] = file_id
            files[position] = size

            file_id += 1
            is_file = False
        else:
            gaps[position] = size

            is_file = True

        position += size

    return memory, files, gaps


parse_input(puzzle.examples[0].input_data)

(array([ 0,  0, -1, -1, -1,  1,  1,  1, -1, -1, -1,  2, -1, -1, -1,  3,  3,
         3, -1,  4,  4, -1,  5,  5,  5,  5, -1,  6,  6,  6,  6, -1,  7,  7,
         7, -1,  8,  8,  8,  8,  9,  9]),
 {0: np.int64(2),
  np.int64(5): np.int64(3),
  np.int64(11): np.int64(1),
  np.int64(15): np.int64(3),
  np.int64(19): np.int64(2),
  np.int64(22): np.int64(4),
  np.int64(27): np.int64(4),
  np.int64(32): np.int64(3),
  np.int64(36): np.int64(4),
  np.int64(40): np.int64(2)},
 {np.int64(2): np.int64(3),
  np.int64(8): np.int64(3),
  np.int64(12): np.int64(3),
  np.int64(18): np.int64(1),
  np.int64(21): np.int64(1),
  np.int64(26): np.int64(1),
  np.int64(31): np.int64(1),
  np.int64(35): np.int64(1),
  np.int64(40): np.int64(0)})

In [4]:
def fragment(memory):
    memory = memory.copy()
    lhs, rhs = 0, len(memory) - 1

    while lhs < rhs:
        if memory[lhs] >= 0:
            lhs += 1
        elif memory[rhs] < 0:
            rhs -= 1
        else:
            memory[lhs] = memory[rhs]
            memory[rhs] = -1

    return memory


def checksum(memory):
    position = np.arange(len(memory))

    checksum = np.where(memory >= 0, position * memory, 0).sum()
    return checksum

fragmented = fragment(parse_input(puzzle.examples[0].input_data)[0])

In [5]:
for example in puzzle.examples:
    print(
        checksum(fragment(parse_input(example.input_data)[0])),
        "-",
        example.answer_a,
    )

1928 - 1928


In [6]:
aocd.submit(
    checksum(fragment(parse_input(puzzle.input_data)[0])),
    part="a",
    year=2024,
    day=9,
)

coerced int64 value np.int64(6332189866718) for 2024/09 to '6332189866718'


aocd will not submit that answer again. At 2024-12-10 13:46:43.027811-05:00 you've previously submitted 6332189866718 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


In [7]:
def defragment_file_level(memory, files, gaps):
    memory = memory.copy()
    gaps = gaps.copy()

    for file_position, file_size in sorted(files.items(), key=lambda x: x[0], reverse=True):
        # Should use an inherently sorted data structure. Well - good enough.
        for gap_position, gap_size in sorted(gaps.items(), key=lambda x: x[0], reverse=False):
            if gap_position > file_position:
                continue

            if gap_size >= file_size:
                file_id = memory[file_position]
                memory[gap_position : gap_position + file_size] = file_id
                memory[file_position : file_position + file_size] = -1
                del gaps[gap_position]

                if gap_size > file_size:
                    gaps[gap_position + file_size] = gap_size - file_size

                break

    return memory


memory, files, gaps = parse_input(puzzle.examples[0].input_data)
defragment_file_level(memory, files, gaps)

array([ 0,  0,  9,  9,  2,  1,  1,  1,  7,  7,  7, -1,  4,  4, -1,  3,  3,
        3, -1, -1, -1, -1,  5,  5,  5,  5, -1,  6,  6,  6,  6, -1, -1, -1,
       -1, -1,  8,  8,  8,  8, -1, -1])

In [8]:
for example in puzzle.examples:
    print(
        checksum(defragment_file_level(*parse_input(example.input_data))),
        "-",
        example.answer_b,
    )

2858 - 2858


In [9]:
checksum(defragment_file_level(*parse_input(puzzle.input_data)))

np.int64(6353648390778)

In [10]:
aocd.submit(
    checksum(defragment_file_level(*parse_input(puzzle.input_data))),
    part="b",
    year=2024,
    day=9,
)

coerced int64 value np.int64(6353648390778) for 2024/09 to '6353648390778'


aocd will not submit that answer again. At 2024-12-10 15:51:20.020571-05:00 you've previously submitted 6353648390778 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 9! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
